In [63]:
import pandas as pd
import numpy as np

from scipy.stats import truncnorm

## Data Import
### Loan Origination Data

In [64]:
loan_colnames =['CREDIT_SCORE', 'FIRST_PAYMENT_DATE', 'FIRST_TIME_HOMEBUYER', 'MATURITY_DATE', 'METROPOLITAN_STATISTICAL_AREA',
 'MORTGAGE_INSURANCE_PERC', 'NUMBER_UNITS','OCCUPANCY_STATUS', 'ORIG_COMBINED_LTV', 'ORIG_DTI_RATIO',
 'ORIG_UPB', 'ORIG_LTV', 'ORIG_INTEREST_RATE', 'CHANNEL', 'PPM', 'AMORTIZATION_TYPE', 'PROPERTY_STATE',
 'PROPERTY_TYPE', 'POSTAL_CODE', 'LOAN_SEQ_NUMBER', 'LOAN_PURPOSE', 'ORIG_LOAN_TERM', 'NUMBER_BORROWERS',
 'SELLER_NAME', 'SERVICER_NAME', 'SUPER_CONFORMING', 'PRE_HARP_LOAN_SEQ_NUMBER', 'PROGRAM_INDICATOR',
 'HARP_INDICATOR', 'PROPERTY_VALUATION_METHOD', 'INTEREST_ONLY']


loan_monthly_cols=['LOAN_SEQ_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_UPB', 'CURRENT_LOAN_DELIQUENCY_STATUS',
 'LOAN_AGE', 'REMAINING_MONHTS_TO_MATURITY', 'REPURCHASED', 'MODIFICATION', 'ZERO_BALANCE_CODE',
 'ZERO_BALANCE_EFFECTIVE_DATE', 'CURRENT_INT_RATE', 'CURRENT_DEFFERRED_UPB', 'DDLPI', 'MI_RECOVERIES',
 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTANANCE_PRESERVATION_COSTS',
 'TAX_AND_INSURANCES', 'MISCELLANEOUS_EXPENSES', 'ACTUAL LOSS CALCULATION', 'MODIFICATION_COST',
 'STEP_MODIFICATION', 'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LTV', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST',
 'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']

In [66]:
# loan origination data from 2010 - 2015
loanlevel_full = pd.read_csv('../data-raw/full_loanlevel_2015.csv')
loanlevel_full.head()

C:\Users\Jan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER,MATURITY_DATE,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERC,NUMBER_UNITS,OCCUPANCY_STATUS,ORIG_COMBINED_LTV,ORIG_DTI_RATIO,...,NUMBER_BORROWERS,SELLER_NAME,SERVICER_NAME,SUPER_CONFORMING,PRE_HARP_LOAN_SEQ_NUMBER,PROGRAM_INDICATOR,HARP_INDICATOR,PROPERTY_VALUATION_METHOD,INTEREST_ONLY,LOAN_ORIG_YEAR
0,784,201005,N,202504,45780.0,0,1,P,90,38,...,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,N,2010
1,795,201003,9,202502,NaN,0,1,P,67,35,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
2,784,201003,N,202502,36100.0,0,1,P,55,47,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
3,759,201003,N,204002,47260.0,0,1,P,50,27,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
4,695,201004,N,204003,NaN,0,1,P,50,21,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010


## Sample Loan IDs for Train & Test Dataset

In [67]:
orig_loan_train = loanlevel_full.copy()
orig_loan_train.FIRST_PAYMENT_DATE = pd.to_datetime(orig_loan_train.FIRST_PAYMENT_DATE, format='%Y%m')

print(orig_loan_train.shape)
orig_loan_train

(300000, 32)


,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER,MATURITY_DATE,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERC,NUMBER_UNITS,OCCUPANCY_STATUS,ORIG_COMBINED_LTV,ORIG_DTI_RATIO,...,NUMBER_BORROWERS,SELLER_NAME,SERVICER_NAME,SUPER_CONFORMING,PRE_HARP_LOAN_SEQ_NUMBER,PROGRAM_INDICATOR,HARP_INDICATOR,PROPERTY_VALUATION_METHOD,INTEREST_ONLY,LOAN_ORIG_YEAR
0,784,2010-05-01,N,202504,45780.0,0,1,P,90,38,...,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,N,2010
1,795,2010-03-01,9,202502,NaN,0,1,P,67,35,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
2,784,2010-03-01,N,202502,36100.0,0,1,P,55,47,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
3,759,2010-03-01,N,204002,47260.0,0,1,P,50,27,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
4,695,2010-04-01,N,204003,NaN,0,1,P,50,21,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,782,2015-12-01,Y,204511,26900.0,0,1,P,97,39,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2015
299996,776,2016-02-01,Y,204601,15940.0,0,1,P,97,34,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2015
299997,791,2015-12-01,N,204511,48700.0,0,1,P,80,38,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2015
299998,704,2016-01-01,Y,204512,39300.0,0,1,P,95,33,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N,2015


## Loan Performance Dataset

### Train Data
+ separate for train and test data due to different handling ob observation time for each loan

### Data Cleaning

In [69]:
## Set the correct types for columns
def preprocess_loan_performance(loan_performance, cutoff_months):
    loan_performance['MONTHLY_REPORTING_PERIOD'] = pd.to_datetime(loan_performance.MONTHLY_REPORTING_PERIOD,format='%Y%m')
    
    # transform loan deliquency as int
    deliquency_mapping = {'R':-1,'XX': np.nan}
    # transform into String
    loan_performance.CURRENT_LOAN_DELIQUENCY_STATUS = loan_performance.CURRENT_LOAN_DELIQUENCY_STATUS.astype('string').replace(deliquency_mapping)
    loan_performance.CURRENT_LOAN_DELIQUENCY_STATUS = loan_performance.CURRENT_LOAN_DELIQUENCY_STATUS.astype('int')
    
    return loan_performance


### Label Data
##### Default Label & Prepayment Label

**only considered for loans before cutoff period...**

In [70]:
# label loans as default, in case they have been delinquent for more than 3 consecutive months
def get_loan_defaults(loan_performance):

    defaults = loan_performance.loc[loan_performance.CURRENT_LOAN_DELIQUENCY_STATUS >= 3, ['LOAN_SEQ_NUMBER','MONTHLY_REPORTING_PERIOD']]
    defaults = defaults.groupby('LOAN_SEQ_NUMBER').min().reset_index()
    # add the loan age at default
    defaults = pd.merge(defaults, loan_performance[['LOAN_SEQ_NUMBER','MONTHLY_REPORTING_PERIOD','LOAN_AGE']], on=['LOAN_SEQ_NUMBER','MONTHLY_REPORTING_PERIOD'])
    defaults = defaults.rename(columns={'MONTHLY_REPORTING_PERIOD':'LAST_OBSERVATION_D','LOAN_AGE':'DEFAULT_TIME'})
    defaults['DEFAULT'] = 1
    
    print('default share: ' + str(len(defaults) / len(loan_performance.LOAN_SEQ_NUMBER.unique()) ))

    defaults.LAST_OBSERVATION_D = pd.to_datetime(defaults.LAST_OBSERVATION_D, format="%Y%m")

    return defaults

# label loans as prepaid, in case the min CURRENT_UPB turned 0
def get_loan_prepayment(loan_performance):

    prepayments = loan_performance.groupby('LOAN_SEQ_NUMBER', as_index=False).agg({'CURRENT_UPB':['min'],
                                                                                     'LOAN_AGE':['max']})
    prepayments.columns = ['LOAN_SEQ_NUMBER', 'MIN_UPB', 'PREPAYMENT_TIME']
    prepayments['PREPAYMENT'] = 1
    prepayments = prepayments[prepayments.MIN_UPB == 0]
    
    prepayments = prepayments.drop('MIN_UPB', axis=1)
    
    print('prepayment share: ' + str(len(prepayments) / len(loan_performance.LOAN_SEQ_NUMBER.unique()) ))

    return prepayments

### Loan Lifetime and Observation Time Sampling
### Feature Engineering
#### Delinquent Periods within 12 months

In [71]:
# draw a sample for a Gaussian truncated dist.
# Gaussian with fixed lower and upper sample bounds
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    if mean == 0:
        return 0 
    
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs()

# sample the observation age for each loan
def gaussian_obs_loan_age(max_loan_age, sd = 10):
    # avg for sampling
    mean_loan_age = int(max_loan_age / 2)
    
    # draw a number within the range
    observation_loan_age = int(get_truncated_normal(mean= mean_loan_age, sd=sd, low=0, upp=max_loan_age))
    
    return observation_loan_age

# for each loan get its lifetime (origination date until turining default, prepayment or latest payment date for an active loan)
# also sample a random time for each loan which is used as the observation time
def get_loans_lifetimes(loan_performance, defaults):

    #for each loan get the start and finish date of observation
    loans_lifetime = loan_performance.groupby('LOAN_SEQ_NUMBER', as_index=False).agg({'MONTHLY_REPORTING_PERIOD':['min','max'],
                                                                                     'LOAN_AGE':['min','max']})
    loans_lifetime.columns = ['LOAN_SEQ_NUMBER', 'START_DATE', 'FINISH_DATE', 'MIN_LOAN_AGE', 'MAX_LOAN_AGE']
    
    # fix the loan origination date as the last date - loan age at that time
    # in some loan origination date is much earlier than date of first payments...
    loans_lifetime['START_DATE'] = loans_lifetime['START_DATE'] - loans_lifetime['MIN_LOAN_AGE'].apply(lambda x: pd.DateOffset(months=x))
    
    loans_lifetime = pd.merge(loans_lifetime, defaults, how='left', on='LOAN_SEQ_NUMBER')

    # in case a lon defaulted set the default date as finish date
    loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'FINISH_DATE'] = loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'LAST_OBSERVATION_D']
    loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'MAX_LOAN_AGE'] = loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'DEFAULT_TIME']

    # drop helper column
    loans_lifetime = loans_lifetime.drop('LAST_OBSERVATION_D', axis=1)
    # convert period to int
    loans_lifetime.DEFAULT = loans_lifetime.DEFAULT.fillna(0)
    loans_lifetime.DEFAULT = loans_lifetime.DEFAULT.astype('int')
    loans_lifetime.DEFAULT_TIME = loans_lifetime.DEFAULT_TIME.fillna(0)
    loans_lifetime.DEFAULT_TIME = loans_lifetime.DEFAULT_TIME.astype('int')
    loans_lifetime.MAX_LOAN_AGE = loans_lifetime.MAX_LOAN_AGE.fillna(0)
    loans_lifetime.MAX_LOAN_AGE = loans_lifetime.MAX_LOAN_AGE.astype('int')
    
    # sample the observation age of the loans
    loans_lifetime['OBSERVATION_LOAN_AGE'] = loans_lifetime.apply(lambda x : gaussian_obs_loan_age(x['MAX_LOAN_AGE'], sd = x['MAX_LOAN_AGE']), axis=1)
    loans_lifetime['OBSERVATION_DATE'] = loans_lifetime.START_DATE + loans_lifetime['OBSERVATION_LOAN_AGE'].apply(lambda x: pd.DateOffset(months=x))
    
    loans_lifetime['REMAINING_SURVIVAL_TIME'] = loans_lifetime.MAX_LOAN_AGE - loans_lifetime.OBSERVATION_LOAN_AGE

    # 12 months cutoff date to determine features within 
    loans_lifetime['12_MONTHS_CUTOFF'] = pd.to_datetime((loans_lifetime.OBSERVATION_DATE.dt.year-1).astype('string') + '-' + loans_lifetime.OBSERVATION_DATE.dt.month.astype('string') + '-01',format='%Y-%m-%d')
    # in case 12 months before initiation set to start date
    loans_lifetime.loc[loans_lifetime['12_MONTHS_CUTOFF'] < loans_lifetime['START_DATE'], '12_MONTHS_CUTOFF'] = loans_lifetime.loc[loans_lifetime['12_MONTHS_CUTOFF'] < loans_lifetime['START_DATE'], 'START_DATE']

    return loans_lifetime



# for each loan get its lifetime (origination date until turining default, prepayment or latest payment date for an active loan)
def get_loans_lifetimes_test(loan_performance, defaults, obs_date = '2019-01-01'):
    
    obs_date = pd.to_datetime(obs_date)
    
    #for each loan get the start and finish date of observation
    loans_lifetime = loan_performance.groupby('LOAN_SEQ_NUMBER', as_index=False).agg({'MONTHLY_REPORTING_PERIOD':['min','max'],
                                                                                     'LOAN_AGE':['min','max']})
    
    loans_lifetime.columns = ['LOAN_SEQ_NUMBER', 'START_DATE', 'FINISH_DATE', 'MIN_LOAN_AGE', 'MAX_LOAN_AGE']
    
    # fix the loan origination date as the last date - loan age at that time
    # in some loan origination date is much earlier than date of first payments...
    loans_lifetime['START_DATE'] = loans_lifetime['START_DATE'] - loans_lifetime['MIN_LOAN_AGE'].apply(lambda x: pd.DateOffset(months=x))
    
    loans_lifetime = pd.merge(loans_lifetime, defaults, how='left', on='LOAN_SEQ_NUMBER')

    # in case a lon defaulted set the default date as finish date
    loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'FINISH_DATE'] = loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'LAST_OBSERVATION_D']
    loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'MAX_LOAN_AGE'] = loans_lifetime.loc[loans_lifetime.DEFAULT == 1, 'DEFAULT_TIME']

    # drop helper column
    loans_lifetime = loans_lifetime.drop('LAST_OBSERVATION_D', axis=1)
    # convert period to int
    loans_lifetime.DEFAULT = loans_lifetime.DEFAULT.fillna(0)
    loans_lifetime.DEFAULT = loans_lifetime.DEFAULT.astype('int')
    loans_lifetime.DEFAULT_TIME = loans_lifetime.DEFAULT_TIME.fillna(0)
    loans_lifetime.DEFAULT_TIME = loans_lifetime.DEFAULT_TIME.astype('int')
    loans_lifetime.MAX_LOAN_AGE = loans_lifetime.MAX_LOAN_AGE.fillna(0)
    loans_lifetime.MAX_LOAN_AGE = loans_lifetime.MAX_LOAN_AGE.astype('int')
    
    # filter out loans for which are not active during observation date
    loans_lifetime = loans_lifetime[loans_lifetime.START_DATE < obs_date]
    loans_lifetime = loans_lifetime[loans_lifetime.FINISH_DATE > obs_date]
    
    
    loans_lifetime['OBSERVATION_DATE'] = obs_date
    loans_lifetime['OBSERVATION_LOAN_AGE'] = (obs_date.year - loans_lifetime.START_DATE.dt.year) * 12 + (obs_date.month - loans_lifetime.START_DATE.dt.month)
    
    
    loans_lifetime['REMAINING_SURVIVAL_TIME'] = loans_lifetime.MAX_LOAN_AGE - loans_lifetime.OBSERVATION_LOAN_AGE

    # 12 months cutoff date to determine features within 
    loans_lifetime['12_MONTHS_CUTOFF'] = pd.to_datetime((loans_lifetime.OBSERVATION_DATE.dt.year-1).astype('string') + '-' + loans_lifetime.OBSERVATION_DATE.dt.month.astype('string') + '-01',format='%Y-%m-%d')
    # in case 12 months before initiation set to start date
    loans_lifetime.loc[loans_lifetime['12_MONTHS_CUTOFF'] < loans_lifetime['START_DATE'], '12_MONTHS_CUTOFF'] = loans_lifetime.loc[loans_lifetime['12_MONTHS_CUTOFF'] < loans_lifetime['START_DATE'], 'START_DATE']

    return loans_lifetime

In [72]:
# get the number of times a loan has been delinquent
def get_delinquency_counts(loan_observed_performance):
    delinquent_counts = loan_observed_performance.loc[loan_observed_performance.MONTHLY_REPORTING_PERIOD > loan_observed_performance['12_MONTHS_CUTOFF'], ['LOAN_SEQ_NUMBER','CURRENT_LOAN_DELIQUENCY_STATUS','START_DATE']].groupby(['LOAN_SEQ_NUMBER','CURRENT_LOAN_DELIQUENCY_STATUS']).count().reset_index()

    delinquent_counts = delinquent_counts.loc[delinquent_counts.CURRENT_LOAN_DELIQUENCY_STATUS >= 0,:]

    delinquent_counts = delinquent_counts.rename(columns={'START_DATE':'COUNT_DELINQUENT_PERIODS'})

    # transform to wide data
    delinquent_counts = delinquent_counts.pivot(index='LOAN_SEQ_NUMBER', columns='CURRENT_LOAN_DELIQUENCY_STATUS', values='COUNT_DELINQUENT_PERIODS')

    # only keep delinquencies up to 60 days
    delinquent_counts = delinquent_counts.loc[: , [0,1,2]]

    delinquent_counts = delinquent_counts.reset_index()
    delinquent_counts = delinquent_counts.fillna(0)

    delinquent_counts = delinquent_counts.rename(columns={0: 'TIMES_NOT_BEING_DELINQUENT', 1:'TIMES_BEING_DELINQUENT_30_DAYS',2:'TIMES_BEING_DELINQUENT_60_DAYS'})

    return delinquent_counts

#### Unpaid Balance as Share

In [73]:
# the unpaid balance share from the performance data
def get_final_upb_share(loan_observed_performance, loanlevel_full):
    # only keep the last observed upb  value
    last_upb = loan_observed_performance.sort_values(['LOAN_SEQ_NUMBER','MONTHLY_REPORTING_PERIOD']).groupby('LOAN_SEQ_NUMBER').tail(1)[['LOAN_SEQ_NUMBER','CURRENT_UPB']]
    last_upb = pd.merge(last_upb, loanlevel_full[['LOAN_SEQ_NUMBER', 'ORIG_UPB']], how='inner', on='LOAN_SEQ_NUMBER')
    # get the share of the total loan volume
    last_upb['UPB_Share'] = last_upb['CURRENT_UPB'] / last_upb['ORIG_UPB']

    return last_upb


# Final preprocessing steps of the output table
def preprocss_labelled_data(data_labelled):
    data_labelled.loc[: , 'DEFAULT_TIME'] = data_labelled.loc[:, 'DEFAULT_TIME'].fillna(0)
    data_labelled.loc[: , 'DEFAULT'] = data_labelled.loc[:, 'DEFAULT'].fillna(0)

    data_labelled.loc[: , 'TIMES_NOT_BEING_DELINQUENT'] = data_labelled.loc[:, 'TIMES_NOT_BEING_DELINQUENT'].fillna(0)
    data_labelled.loc[: , 'TIMES_BEING_DELINQUENT_30_DAYS'] = data_labelled.loc[:, 'TIMES_BEING_DELINQUENT_30_DAYS'].fillna(0)
    data_labelled.loc[: , 'TIMES_BEING_DELINQUENT_60_DAYS'] = data_labelled.loc[:, 'TIMES_BEING_DELINQUENT_60_DAYS'].fillna(0)

    # rearrange and delete columns
    data_labelled = data_labelled[['LOAN_SEQ_NUMBER', 'CREDIT_SCORE', 'FIRST_PAYMENT_DATE', 'LOAN_ORIG_YEAR','FIRST_TIME_HOMEBUYER',
           'MATURITY_DATE', 'METROPOLITAN_STATISTICAL_AREA',
           'MORTGAGE_INSURANCE_PERC', 'NUMBER_UNITS', 'OCCUPANCY_STATUS',
           'ORIG_COMBINED_LTV', 'ORIG_DTI_RATIO', 'ORIG_UPB', 'ORIG_LTV',
           'ORIG_INTEREST_RATE', 'CHANNEL', 'PPM', 'AMORTIZATION_TYPE',
           'PROPERTY_STATE', 'PROPERTY_TYPE',
           'LOAN_PURPOSE', 'ORIG_LOAN_TERM', 'NUMBER_BORROWERS', 'SUPER_CONFORMING',
           'PROPERTY_VALUATION_METHOD',
           'INTEREST_ONLY', 'OBSERVATION_DATE','OBSERVATION_LOAN_AGE', 
           'TIMES_NOT_BEING_DELINQUENT', 'TIMES_BEING_DELINQUENT_30_DAYS', 
           'TIMES_BEING_DELINQUENT_60_DAYS', 'UPB_Share',
           'DEFAULT_TIME', 'DEFAULT','PREPAYMENT','PREPAYMENT_TIME','REMAINING_SURVIVAL_TIME']]

    data_labelled.DEFAULT = data_labelled.DEFAULT.astype('int')
    data_labelled.DEFAULT_TIME = data_labelled.DEFAULT_TIME.astype('int')
    data_labelled.REMAINING_SURVIVAL_TIME = data_labelled.REMAINING_SURVIVAL_TIME.astype('int')
    
    data_labelled.TIMES_NOT_BEING_DELINQUENT = data_labelled.TIMES_NOT_BEING_DELINQUENT.astype('int')
    data_labelled.TIMES_BEING_DELINQUENT_30_DAYS = data_labelled.TIMES_BEING_DELINQUENT_30_DAYS.astype('int')
    data_labelled.TIMES_BEING_DELINQUENT_60_DAYS = data_labelled.TIMES_BEING_DELINQUENT_60_DAYS.astype('int')
    
    return data_labelled

## Run for all Years

In [74]:
loan_monthly_cols=['LOAN_SEQ_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_UPB', 'CURRENT_LOAN_DELIQUENCY_STATUS',
 'LOAN_AGE', 'REMAINING_MONHTS_TO_MATURITY', 'REPURCHASED', 'MODIFICATION', 'ZERO_BALANCE_CODE',
 'ZERO_BALANCE_EFFECTIVE_DATE', 'CURRENT_INT_RATE', 'CURRENT_DEFFERRED_UPB', 'DDLPI', 'MI_RECOVERIES',
 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTANANCE_PRESERVATION_COSTS',
 'TAX_AND_INSURANCES', 'MISCELLANEOUS_EXPENSES', 'ACTUAL LOSS CALCULATION', 'MODIFICATION_COST',
 'STEP_MODIFICATION', 'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LTV', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST',
 'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']

years = [2010, 2011, 2012, 2013, 2014] 

cutoff_months = 48

for year in years:
    
    print('Year: '+ str(year))
    # only sampled loans from that period
    sample_ids = loanlevel_full[loanlevel_full.LOAN_ORIG_YEAR == year].LOAN_SEQ_NUMBER
    
    loan_performance = pd.read_csv('sample_'+str(year)+'/sample_svcg_'+str(year)+'.txt', delimiter="|",names=loan_monthly_cols, header=None)
    # only keep performance data which we samled
    loan_performance = loan_performance[loan_performance.LOAN_SEQ_NUMBER.isin(sample_ids)]
    
    loan_performance = preprocess_loan_performance(loan_performance, cutoff_months)
    # only keep performance data for loans before the cutoff periods
    loan_performance = loan_performance[loan_performance.LOAN_AGE <= cutoff_months]
    
    defaults = get_loan_defaults(loan_performance)
    prepayments = get_loan_prepayment(loan_performance)
    
    #for each loan get the start and finish date of observation
    loans_lifetime = get_loans_lifetimes(loan_performance, defaults)
    
    # append the loan lifetime and observation period
    # includes default label
    loan_performance = pd.merge(loan_performance, loans_lifetime, on='LOAN_SEQ_NUMBER', how='left')
    
    loan_performance = pd.merge(loan_performance, prepayments, on='LOAN_SEQ_NUMBER', how='left')
    # fill NAs in case there is no prepayment
    loan_performance['PREPAYMENT'] = loan_performance['PREPAYMENT'].fillna(0) 
    loan_performance['PREPAYMENT_TIME'] = loan_performance['PREPAYMENT_TIME'].fillna(0) 
    
    # only keep observed loan performance
    loan_observed_performance  = loan_performance[loan_performance.MONTHLY_REPORTING_PERIOD <= loan_performance.OBSERVATION_DATE ]
#
    delinquent_counts = get_delinquency_counts(loan_observed_performance)
    
    last_upb = get_final_upb_share(loan_observed_performance, loanlevel_full)
    
    # only use the loans for which we have performance data of that year...
    data_labelled = pd.merge(orig_loan_train, loans_lifetime[['LOAN_SEQ_NUMBER', 'OBSERVATION_DATE', 'OBSERVATION_LOAN_AGE','REMAINING_SURVIVAL_TIME']], how='inner', on ='LOAN_SEQ_NUMBER')
    # append the prepayment labels
    data_labelled = pd.merge(data_labelled, prepayments, on ='LOAN_SEQ_NUMBER', how='left')
    data_labelled['PREPAYMENT'] = data_labelled['PREPAYMENT'].fillna(0) 
    data_labelled['PREPAYMENT_TIME'] = data_labelled['PREPAYMENT_TIME'].fillna(0) 
    
    data_labelled = pd.merge(data_labelled, defaults[['LOAN_SEQ_NUMBER', 'DEFAULT_TIME', 'DEFAULT']], on ='LOAN_SEQ_NUMBER', how='left')
    
    data_labelled = pd.merge(data_labelled, delinquent_counts, on ='LOAN_SEQ_NUMBER', how='left')
    
    data_labelled = pd.merge(data_labelled, last_upb[['LOAN_SEQ_NUMBER','UPB_Share']], on ='LOAN_SEQ_NUMBER', how='left')
    # in case UPB NaN -> no payment received == full amount still needs to be paid...
    data_labelled['UPB_Share'] = data_labelled['UPB_Share'].fillna(1)
    
    data_labelled = preprocss_labelled_data(data_labelled)
    
    # in case first period, overwrite csv
    if year == years[0]:
        data_labelled.to_csv('../data/loans_labelled_train.csv', index=False, mode='w')
    else:
        data_labelled.to_csv('../data/loans_labelled_train.csv', index=False, mode='a', header=False)


Year: 2010


C:\Users\Jan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,23,24,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


default share: 0.014140521431727796
prepayment share: 0.48276623950508174


C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Year: 2011


...

C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Year: 2014


C:\Users\Jan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


default share: 0.013527134908475123
prepayment share: 0.4062575598742037


C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Test Data

+ same procedure, but here we do not sample the observation time, but it is fixed..

+ consider only loans which are still active during the observation date

+ loans from 2016 - 2018 (48 months)
     + -> observation date: 2019-01-01 (reconstruction of features)
     + 48 months of performance data under consideration
     
+ omit a loan in case it is not active at observation time any more...

In [78]:
loan_monthly_cols=['LOAN_SEQ_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_UPB', 'CURRENT_LOAN_DELIQUENCY_STATUS',
 'LOAN_AGE', 'REMAINING_MONHTS_TO_MATURITY', 'REPURCHASED', 'MODIFICATION', 'ZERO_BALANCE_CODE',
 'ZERO_BALANCE_EFFECTIVE_DATE', 'CURRENT_INT_RATE', 'CURRENT_DEFFERRED_UPB', 'DDLPI', 'MI_RECOVERIES',
 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTANANCE_PRESERVATION_COSTS',
 'TAX_AND_INSURANCES', 'MISCELLANEOUS_EXPENSES', 'ACTUAL LOSS CALCULATION', 'MODIFICATION_COST',
 'STEP_MODIFICATION', 'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LTV', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST',
 'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']

years = [2015]

cutoff_months = 48

observation_date = '2016-01-01'

for year in years:
    
    print('Year: '+ str(year))
    # only sampled loans from that period
    sample_ids = loanlevel_full[loanlevel_full.LOAN_ORIG_YEAR == year].LOAN_SEQ_NUMBER
    
    loan_performance = pd.read_csv('sample_'+str(year)+'/sample_svcg_'+str(year)+'.txt', delimiter="|",names=loan_monthly_cols, header=None)
    # only keep performance data which we samled
    loan_performance = loan_performance[loan_performance.LOAN_SEQ_NUMBER.isin(sample_ids)]
    
    loan_performance = preprocess_loan_performance(loan_performance, cutoff_months)
    # only keep performance data for loans before the cutoff periods
    loan_performance = loan_performance[loan_performance.LOAN_AGE <= cutoff_months]
    
    defaults = get_loan_defaults(loan_performance)
    prepayments = get_loan_prepayment(loan_performance)
    
    #for each loan get the start and finish date of observation
    loans_lifetime = get_loans_lifetimes_test(loan_performance, defaults, obs_date=observation_date)
    
    # append the loan lifetime and observation period
    # includes default label
    loan_performance = pd.merge(loan_performance, loans_lifetime, on='LOAN_SEQ_NUMBER', how='left')
    
    loan_performance = pd.merge(loan_performance, prepayments, on='LOAN_SEQ_NUMBER', how='left')
    # fill NAs in case there is no prepayment
    loan_performance['PREPAYMENT'] = loan_performance['PREPAYMENT'].fillna(0) 
    loan_performance['PREPAYMENT_TIME'] = loan_performance['PREPAYMENT_TIME'].fillna(0) 
    
    # only keep observed loan performance
    loan_observed_performance  = loan_performance[loan_performance.MONTHLY_REPORTING_PERIOD <= loan_performance.OBSERVATION_DATE ]
#
    delinquent_counts = get_delinquency_counts(loan_observed_performance)
    
    last_upb = get_final_upb_share(loan_observed_performance, loanlevel_full)
    
    # only use the loans for which we have performance data of that year...
    data_labelled = pd.merge(orig_loan_train, loans_lifetime[['LOAN_SEQ_NUMBER', 'OBSERVATION_DATE', 'OBSERVATION_LOAN_AGE','REMAINING_SURVIVAL_TIME']], how='inner', on ='LOAN_SEQ_NUMBER')
    
    data_labelled = data_labelled[data_labelled.FIRST_PAYMENT_DATE < observation_date]
    
    # append the prepayment labels
    data_labelled = pd.merge(data_labelled, prepayments, on ='LOAN_SEQ_NUMBER', how='left')
    data_labelled['PREPAYMENT'] = data_labelled['PREPAYMENT'].fillna(0) 
    data_labelled['PREPAYMENT_TIME'] = data_labelled['PREPAYMENT_TIME'].fillna(0) 
    
    data_labelled = pd.merge(data_labelled, defaults[['LOAN_SEQ_NUMBER', 'DEFAULT_TIME', 'DEFAULT']], on ='LOAN_SEQ_NUMBER', how='left')
    
    data_labelled = pd.merge(data_labelled, delinquent_counts, on ='LOAN_SEQ_NUMBER', how='left')
    
    data_labelled = pd.merge(data_labelled, last_upb[['LOAN_SEQ_NUMBER','UPB_Share']], on ='LOAN_SEQ_NUMBER', how='left')
    # in case UPB NaN -> no payment received == full amount still needs to be paid...
    data_labelled['UPB_Share'] = data_labelled['UPB_Share'].fillna(1)
    
    data_labelled = preprocss_labelled_data(data_labelled)
    
    # in case first period, overwrite csv
    if year == years[0]:
        data_labelled.to_csv('../data/loans_labelled_test.csv', index=False, mode='w')
    else:
        data_labelled.to_csv('../data/loans_labelled_test.csv', index=False, mode='a', header=False)


Year: 2015


C:\Users\Jan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


default share: 0.011211980917490288
prepayment share: 0.3223092252259506


C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\arrays\datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,
C:\Users\Jan\AppData\Roaming\Python\Python36\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
